# Stochastic Nonparametric Envelopment of Data (StoNED)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

### Objective: Estimates a log-transformed cost function model and technicial inefficiency

In [2]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Prepare and read data

Download the Excel data from <https://github.com/ds2010/StoNED-Python/tree/master/Data>

Data description is now available at <https://github.com/ds2010/StoNED-Python/blob/master/Data/data_description.ipynb>

In [3]:
# import data
df = pd.read_excel ('data.xlsx')

In [4]:
# output (total cost)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

Note 1: Converting Numpy array to list by `array.tolist()` can speed up the computation.

In [5]:
# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

### Stage 1: Convex Nonparametric Least Squares estimation

In [6]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

crt   = "mult"
func  = "cost"
pps   = "crs"

model = CNLS.cnls(y, x, crt, func, pps)
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

Note 2:
     # crt     = "addi" : Additive composite error term
     #         = "mult" : Multiplicative composite error term
     # func    = "prod" : production frontier
     #         = "cost" : cost frontier
     # pps     = "vrs"  : variable returns to scale production possibility sets (pps)
     #         = "crs"  : constant returns to scale pps

In [7]:
# display estimates
model.e.display() 
model.b.display() 

e : residuals
    Size=89, Index=i
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :  None :    0.03591704716740125 :  None : False : False :  Reals
      1 :  None :   0.026035484499695317 :  None : False : False :  Reals
      2 :  None :     0.2021966194208569 :  None : False : False :  Reals
      3 :  None :  -0.004517966599345962 :  None : False : False :  Reals
      4 :  None :    0.17430052203661542 :  None : False : False :  Reals
      5 :  None :   -0.11912248827552518 :  None : False : False :  Reals
      6 :  None :   -0.01130133790753906 :  None : False : False :  Reals
      7 :  None :    0.11921720099488699 :  None : False : False :  Reals
      8 :  None :     0.3736984392624628 :  None : False : False :  Reals
      9 :  None :  -0.040184225000433275 :  None : False : False :  Reals
     10 :  None :    0.02303955229234552 :  None : False : False :  Reals
     11 :  None :   -0.07779539297949795 :  None : False : False :  Reals
   

In [8]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

### Stage 2: Residuals decomposition

In [9]:
# decomposition by method od moment (MoM)
method      = 'MoM'
TE          = StoNED.stoned(eps, func, method, crt)

Note 3:

     # func    = "prod": production frontier;  
     #         = "cost": cost frontier
     # method  = "MOM" : Method of moments
     #         = "QLE" : Quasi-likelihood estimation 
     #         = "NKD" : Nonparametric kernel deconvolution (TBA...)
     # crt     = "addi": Additive composite error term
     #         = "mult": Multiplicative composite error term

In [10]:
# display technicial inefficiency
TE

array([1.04136199, 1.04711176, 0.94921548, 1.06509152, 0.99433259,
       1.13532725, 1.06912502, 1.00482688, 0.8626986 , 1.08647094,
       1.04886127, 1.10948124, 0.92534107, 0.91931829, 1.07739518,
       1.15154967, 0.95315927, 1.00006759, 1.07064014, 1.12249307,
       1.14029974, 1.2283932 , 0.95942853, 1.11956764, 1.00453605,
       1.02470228, 0.92751277, 1.18164352, 1.00731073, 1.04613028,
       1.06333665, 1.35091413, 1.08944972, 0.94196072, 1.06479589,
       1.00195162, 1.19825458, 1.15674835, 1.13275316, 1.08457313,
       1.09136593, 1.03602752, 1.02465658, 1.09966129, 1.11657705,
       1.14276014, 1.04636001, 1.01467671, 1.14689372, 1.14387916,
       0.95253125, 1.07595357, 0.9501686 , 0.95044345, 1.03733446,
       1.19961598, 1.20750223, 1.11619072, 1.18128735, 1.05050252,
       1.19707977, 1.11107963, 1.08379545, 1.05326294, 0.86643668,
       1.12926062, 1.01600102, 1.09173278, 1.01554781, 1.21397746,
       1.09162706, 1.0160282 , 1.18296252, 1.12100284, 1.11564